# **From stellar stream/DM subhalos encounters to power spectrum: a one way analysis**

# Description

Like the stream_sim_galpy notebook, this notebook is meant to simulate stellar streams formed through star cluster accretion in a host potential, and have the same dependencies, apart from the galpy extension streampepperdf.py which allow the user to perform multiple DMS impact for one stellar stream pdf generation. To use it, the user must DL the python extension at https://gist.github.com/jobovy/1be0be25b525e5f50ea3 and add it to the galpy df folder, update the \_init\_.py file and change:

 "from galpy.util import bovy_conversion, bovy_coords" (because it is deprecated) to\
 "import galpy.util.conversion as bovy_conversion\
  import galpy.util.coords as bovy_coords"

It requires a config files, found in config folder, with particular cluster data (Pal_5 config fle is given as an example). User can create his own sts config file from the custom_config file in the same directory.
It is paired with simulation/data analysis python files found in Codes directory.
It switches beetween ICRS, galactic and (phi1, phi2) coordinate systems, depending of the user specification.

Required libraries to make it work are specified in the README file.

Libraries

In [8]:
###Libraries
#Mains
%matplotlib widget
# %matplotlib inline
import galpy.potential as gp
import galpy.df as gd #for streams PDF generation
import galpy.actionAngle as ga
from galpy.orbit import Orbit

#Utils
from importlib import reload
import pandas as pd
import sys
import yaml
import os
#Define user path
sys.path.append(os.path.abspath('../'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import astropy.units as u
import astropy.coordinates as ac
from astropy.table import Table
# import galpy.util.conversion as guconv
import galpy.util.coords as gucoord
import galpy.util.conversion as guconv
import gala.coordinates as gc #for great circle rotation
with ac.galactocentric_frame_defaults.set("v4.0"):
    galcen_frame = ac.Galactocentric()

In [9]:
###Librairires for simulation and 
import stream_galsim.stream_utils as sutils; reload(sutils) #custom .py file with utilities
from galpy.df import streampepperdf #Extension to perform multiple impact
import stream_galsim.impact_distribution as sid; reload(sid)


<module 'stream_galsim.impact_distribution' from '/home/hallouin/Documents/thall_2025/Streams/stream_sim/stream_galsim/impact_distribution.py'>

# Power spectrum of one stellar stream

The goal here is to compute a power spectrum of a simulated stellar stream, by mimicking as much as we can the properties of an existing stellar stream.

At first we set the properties of the unperturbed system, including a progenitor (usually a globular cluster) and an accreting host in which it is stripped.

## Initialization

In [3]:
##Load progenitor config file
# Path to the yaml config file in the 'config' folder
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
config_file = 'galstreams_Pal5_config.yaml'
config_path = os.path.join(base_dir, 'config', config_file)

# Load YAML and convert it to a DataFrame
with open(config_path, 'r') as file:
    config_data = yaml.safe_load(file)

##Display cluster data
#current pose/velocity in ICRS
stream_data = pd.DataFrame(config_data)
# print(stream_data)
#only progenitor data is wanted for our simulation
progenitor = stream_data[4:].loc[:,["cluster","ra","dec","distance","pm_ra_cosdec","pm_dec","radial_velocity"]]
#["dec","dec_unit","distance","distance_unit","frame","pm_dec","pm_dec_unit","pm_ra_cosdec","pm_ra_cosdec_unit", "ra","ra_unit","radial_velocity","radial_velocity_unit"]
print(progenitor.values.tolist()[0])

['Pal5', 229.0, -0.124, 22.9, -2.296, -2.257, -58.7]


In [4]:
###Progenitor
#progenitor coord 
cluster_name = progenitor.values.tolist()[0][0] #name
prog_orbit = Orbit(progenitor.values.tolist()[0][1:], radec=True) #progenitor orbit instance. radec=True specify the frame used radec=ICRS


prog_mass = 2*10.**4.*u.Msun
prog_a = 20.*u.pc #scale radius of the plummer sphere
rc = 23*u.pc #progenitor radius
prog_pot = gp.PlummerPotential(prog_mass, prog_a)#Define progenitor with a plummer potential
prog_sigv = 0.365 *u.km/u.s
t_disrupt = 4.5*u.Gyr #time of disruption of cluster, typically 4.5Gyr
# tdisrupt= guc.time_in_Gyr(V0,R0)

In [5]:
###Accreting host
### Usually Milky way potential
V0, R0= 245.6, 8.122 #scale parameters
vsun=[-12.9,245.6,7.78] #for streamdf, vxyz
mw_pot = gp.MWPotential2014
aaisochrone = ga.actionAngleIsochroneApprox(pot=mw_pot,b=1.5)

Simulating the unperturbed model

In [6]:
sid.NonPerturbedStreamModel(sigv = prog_sigv,
            progenitor=prog_orbit,
            pot=mw_pot,
            aA=aaisochrone,
            leading=True,
            ro = R0, #Distance scale for translation into internal units
            vsun=vsun,
            tdisrupt=t_disrupt)

AttributeError: module 'stream_galsim.impact_distribution' has no attribute 'NonPerturbedStreamModel'

## Statistical sampling of a DM subhalos distribution

We want to match as much as we can the simulation to real data, thus we need to set a realistic environment of creation of a perturbed stream. Already having the progenitor and an accreting host, we miss an accurate distribution of DMS in the accreting host. Thus we list the impact properties:
- the impact time ti.
- the stream parallel angle at which the impact occurs at the time of closest approach.
- the fly-by velocity of the dark matter halo.
- the impact parameter
- the intrinsic properties of the subhalo (mass, density profile and radius)

Those properties 

In [5]:
### Impact times